# Approx Lab

The goal of this lab is to gain experience using approximation algorithms.

For the purpose of this assignment, you will be implementing the knapsack problem. Before continuing, the following cell contains any dependencies you may need for this assignment.

In [22]:
import pandas as pd

## Background Info

### Knapsack Problem
We are actually going to be looking at a specific (hopefully fun) implementation of the knapsack problem.

The traditional knapsack problem asks you to pack a set of items, with given weights and values into a "knapsack" with a maximum capacity. You cannot pack more items than the capacity of the knapsack. The goal is to select a subset of items that maximize the total value with what can fit in the knapsack. The name "knapsack" comes from the similarity to a hiker trying to pack a bag without going over a certain weight.

The knapsack problem is an NP-Hard optimization problem, which is why it is one of the classic approximation algorithms.

### Specifications for this Lab
The variation that we will be doing in this assignment is referred to as an 0/1 knapsack problem, meaning that items must be completely put in the knapsack or they must be left behind (you can't have a fractional portion of an item). We will be representing this as a simple fantasy sports league - specifically Formula 1 drivers. 

You are provided with a list of F1 drivers and constructors with given costs and points as well as a budget of $100M. You will be selecting five drivers and two constructors (teams). 

### Scoring Information
Points are gained from qualification results for drivers and constructors as well as race results for drivers and constructors. 

<!-- Qualifications
- Drivers score based on their resulting position 
- Constructor score the combined total of their two drivers in qualifying plus a modifier based on how far both their drivers get. -->
<!-- 
| Drivers    | Points |
| -------- | ------- |
| Pole Position (1st Place) | 10 |
| 2nd Place | 9 |
| 3rd Place | 8 |
| 4th Place | 7 |
| 5th Place | 6 |
| 6th Place | 5 |
| 7th Place | 4 |
| 8th Place | 3 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| No Time Set | 0 |
| Disqualification | 0 | -->


<!-- | Constructors | Points |
| ----------- | ------- |
| Neither Driver Reaches Q2 | -1 |
| One Driver Reaches Q2 | 1 |
| Both Drivers Reach Q2 | 5 |
| Both Drivers Reach Q3 | 10 | -->


<!-- Race
- Drivers score based on their resulting position, position difference from starting order, and some additional bonuses
- Constructors score a combined total of their two drivers' results (excluding the Driver of the Day bonus) ~~plus a modifier based on pitstop times~~ -->

<!-- | Drivers | Points |
| -------- | ------- |
| Positions Gained | 1/position |
| Positions Lost | -1/position |
| Overtakes Made | 1/overtake |
| Fastest Lap | 10 |
| Driver of the Day | 10 | -->

<!-- | Race Result | Points |
| -------- | ------- |
| 1st Place | 25 |
| 2nd Place | 18 |
| 3rd Place | 15 |
| 4th Place | 12 |
| 5th Place | 10 |
| 6th Place | 8 |
| 7th Place | 6 |
| 8th Place | 4 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| DNF | -20 |
| Disqualification | -20 | -->

Fortunately for you, we have access to the current point standings post-2025 China GP for drivers & constructors, which are available in `drivers.csv` & `constructors.csv`.

*Note: The costs are in millions, so 30 is actually 30 M.

### Questions (Answered in Various Parts of the Lab): 
1. What is the general setup of the problem?
2. Determining the mathematical functions of the problem
3. Implementing a greedy algorithm


## A. Assessing the Problem

The values that are provided for this lab are from the points distribution PRIOR to the Japan Grand Prix (which means we can actually compare with the race results at the submission deadline!)

Create a fantasy team https://fantasy.formula1.com/en/create-team for the upcoming Grand Prix to get a better idea of what drivers, constructors, points, and the budget are. Who is on your team & how much did you spend? Don't spend too much time on this, it really is just to get familiar with the format.

Lewis Hamilton,  Alexander Albon, & Max Verstappen are on my team.
Mercedes and Haas are my constructors.
I spent 97.4 M.

What represents the "items" in the knapsack? 

The constructors and drivers are items in the knapsack.

What represents the weights?

The weights are represents by the cost.

What represents the values?

The points for the season are the values.

In [23]:
# Load the data from the CSV files into a format that you are comfortable working with
# This can be arrays, dictionaries, or pd DataFrames
drivers = pd.read_csv("drivers.csv")
print(drivers)
constructors = pd.read_csv("constructors.csv")
print()
print("-----------------------------------------")
print()
print(constructors)
# Print/Display your data as well to confirm that you have successfully saved it.

                   Driver  Points  Cost
0            Lando Norris     100  29.6
1   Andrea Kimi Antonelli      61  19.3
2          George Russell      60  21.4
3          Max Verstappen      59  28.6
4           Oscar Piastri      55  23.0
5            Lance Stroll      33   9.3
6            Esteban Ocon      32   8.1
7          Alexaner Albon      28  12.8
8         Nico Hulkenberg      22   7.6
9          Oliver Bearman      22   6.7
10         Lewis Hamilton       3  23.6
11           Yuki Tsunoda       0  16.8
12            Liam Lawson       0   8.4
13        Charles LeClerc       0  25.3
14           Pierre Gasly      -7  10.6
15           Isack Hadjar      -9   5.0
16            Jack Doohan      -9   6.0
17           Carlos Sainz     -11  11.9
18      Gabriel Bortoleto     -11   4.8
19        Fernando Alonso     -36   7.6

-----------------------------------------

       Constructor  Points  Cost
0          McLaren     172  30.6
1         Mercedes     136  23.3
2  Red Bull Racin

# B. Integer Program

Before implementing our approx algorithm, we need to first consider the integer program. 

Given the following variables, create an integer program that maximizes the total points of drivers while remaining under the budget: 
- $x_i$ is the decision variable that determines whether driver $i$ is included in the team or not
- $p_i$ is the # of points that driver $i$ is worth
- $c_i$ is the cost of driver $i$

For now, we will focus on just the drivers and ignore the constructors. We will also temporarily reduce our budget to **$50M**

While we are not planning on implementing the full integer program, it helps us quantify parts of the problem

Let $I$ represent the total number of drivers.\
$\text{maximize} \quad \sum_{i = 0}^{I}x_ip_i$  \
$\text{subject to} \
\quad \sum_{i = 0}^{I}x_ic_i \leq 50\ 
\quad \sum_{i = 0}^{I}x_i \leq 5\
\quad x_i \in {0, 1}\
$ 


## Decision Variable & Constraints

We can make helper functions for our future algorithm based on constraints that we derive when forming the IP: 
1. Which drivers should be included in our solution? 
2. What is our maximum cost?
3. What is our maximum # of drivers? 

In [24]:
budget = 50 # in millions

max_drivers = 5
# Replace with your code

# C. Greedy Algorithm


There are multiple ways we can go about solving the knapsack problem. For this lab, we will be focusing on a greedy algorithm and a variant of that.

Let's start with a basic greedy algorithm:
1. Calculate the value-to-cost ratio for each driver 
2. Sort drivers in non-increasing order of value-to-cost
2. Greedily select drivers from list until either budget or max drivers constraint is hit

Use the helper methods from the previous part in your implementation.

In [25]:
def value_to_cost(items): 
    items['VC_Ratio']= items['Points'] / items['Cost']
    return items

def sort_items(items):
    items.sort_values(by='VC_Ratio', ascending = False, inplace=True)
    return items

def greedy_driver_select(drivers):
    drivers = sort_items(value_to_cost(drivers))
    cost = 0
    count = 0
    x = []

    for _, row in drivers.iterrows():
        if cost + row["Cost"] <= budget and count < max_drivers:
            x.append(1)
            cost += row["Cost"]
            count += 1
        else:
            x.append(0)

    drivers["x"] = x
    return drivers
drivers_greedy = drivers.copy()
print(greedy_driver_select(drivers_greedy))

                   Driver  Points  Cost  VC_Ratio  x
6            Esteban Ocon      32   8.1  3.950617  1
5            Lance Stroll      33   9.3  3.548387  1
0            Lando Norris     100  29.6  3.378378  1
9          Oliver Bearman      22   6.7  3.283582  0
1   Andrea Kimi Antonelli      61  19.3  3.160622  0
8         Nico Hulkenberg      22   7.6  2.894737  0
2          George Russell      60  21.4  2.803738  0
4           Oscar Piastri      55  23.0  2.391304  0
7          Alexaner Albon      28  12.8  2.187500  0
3          Max Verstappen      59  28.6  2.062937  0
10         Lewis Hamilton       3  23.6  0.127119  0
11           Yuki Tsunoda       0  16.8  0.000000  0
12            Liam Lawson       0   8.4  0.000000  0
13        Charles LeClerc       0  25.3  0.000000  0
14           Pierre Gasly      -7  10.6 -0.660377  0
17           Carlos Sainz     -11  11.9 -0.924370  0
16            Jack Doohan      -9   6.0 -1.500000  0
15           Isack Hadjar      -9   5.0 -1.800

Try running your basic greedy algorithm. 

You may notice a few flaws:
1. The greedy algorithm is only looking at value-to-cost ratio, which may miss out on an overall improvement in value that still fits within the budget
2. Depending on your implementation, you may not currently be meeting the "5 total drivers" constraint - this is fine


## Redux Greedy Algorithm
We are going to next implement a small adjustment to this algorithm in order to improve its performance. \
This is a 2-approximation for the knapsack problem. 

1. Calculate the value-to-cost ratio for each driver
2. Sort drivers in non-increasing order of value-to-cost
3. Greedily add items until we hit an item that is too big
4. Compare the too big item against the already existing list of items & pick the better of the two

It might help to make a separate function to handle #4. 

In [26]:

def redux_greedy_driver_select(drivers):
    drivers = sort_items(value_to_cost(drivers))
    cost = 0
    val = 0
    count = 0
    x = []

    for idx, row in drivers.iterrows():
        if count < max_drivers:
            if cost + row["Cost"] <= budget:
                x.append(1)
                cost += row["Cost"]
                val += row["Points"]
                count += 1
            else:
                if (val < row["Points"] and row["Cost"] <= budget ):
                    x = [0 for _ in range(idx)]
                    x[idx] = 1              # pick only this driver
                    cost = row["Cost"]
                    val = row["Points"]
                    count = 1
                    break
                else:
                    x.append(0)
        else:
            x.append(0)

    drivers["x"] = x
    return drivers

redux_drivers = drivers.copy()
print(redux_greedy_driver_select(redux_drivers))

                   Driver  Points  Cost  VC_Ratio  x
6            Esteban Ocon      32   8.1  3.950617  1
5            Lance Stroll      33   9.3  3.548387  1
0            Lando Norris     100  29.6  3.378378  1
9          Oliver Bearman      22   6.7  3.283582  0
1   Andrea Kimi Antonelli      61  19.3  3.160622  0
8         Nico Hulkenberg      22   7.6  2.894737  0
2          George Russell      60  21.4  2.803738  0
4           Oscar Piastri      55  23.0  2.391304  0
7          Alexaner Albon      28  12.8  2.187500  0
3          Max Verstappen      59  28.6  2.062937  0
10         Lewis Hamilton       3  23.6  0.127119  0
11           Yuki Tsunoda       0  16.8  0.000000  0
12            Liam Lawson       0   8.4  0.000000  0
13        Charles LeClerc       0  25.3  0.000000  0
14           Pierre Gasly      -7  10.6 -0.660377  0
17           Carlos Sainz     -11  11.9 -0.924370  0
16            Jack Doohan      -9   6.0 -1.500000  0
15           Isack Hadjar      -9   5.0 -1.800

# D. Reintroducing Constructors

We've been neglecting our constructors so far and cutting our budget. Let's make them feel included in our processes again. 

## Integer Program

We should start by redefining our original IP to see what constraints we need to add. \
We've added some more variables to our IP: 
- $x_j$ is the decision variable that determines whether constructor $j$ is included in the team or not 
- $p_j$ is the # of points that constructor $j$ is worth
- $c_j$ is the cost of constructor $j$

Our budget has also been increased back to **100M**. 


$\text{maximize} \quad \sum_{i = 0}^{I}x_ip_i + \sum{j = 0}^{J}x_jp_j$  \
$\text{subject to}\\
 \quad \sum_{i = 0}^{I}x_ic_i + \sum_{j = 0}^{J}x_jc_j \leq 100 \\
 \quad \sum_{i=0}^{I}x_i \leq 5 \\
 \quad \sum_{j=0}^{J}x_j \leq 2 
 \quad \forall i, x_i \in {0, 1}
\quad \forall j, x_j \in {0, 1}\
$

## Constraints 

To redefine our greedy algorithm to include constructors, we have to also write helpers that correspond to our new constraints.

In [27]:
budget = 100 # in millions

max_constructors = 2
greedy_constructors = constructors.copy()
print(sort_items(value_to_cost(greedy_constructors)))

# Replace with your code


       Constructor  Points  Cost  VC_Ratio
3             Haas      61   8.2  7.439024
1         Mercedes     136  23.3  5.836910
0          McLaren     172  30.6  5.620915
5     Racing Bulls      32   8.0  4.000000
2  Red Bull Racing      86  25.4  3.385827
4         Williams      36  13.5  2.666667
6      Kick Sauber      15   6.2  2.419355
8     Aston Martin       5   7.3  0.684932
7          Ferrari      13  27.1  0.479705
9           Alpine     -20   8.3 -2.409639


## Greedy Algorithm

Update the greedy algorithm to calculate for buying both constructors and drivers. \
Remember, your list of non-increasing value-to-cost ratios should be a combination of both drivers & constructors. 

In [31]:
def greedy_f1(drivers, constructors, budget, max_drivers, max_constructors):
    # Combine drivers and constructors
    all_items = pd.concat([drivers, constructors])
    
    # Sort items based on value-to-cost ratio
    all_items = sort_items(value_to_cost(all_items))
    
    cost = 0
    val = 0
    selected_drivers = 0
    selected_constructors = 0
    x = []

    for idx, row in all_items.iterrows():
        # Check if it's a driver or a constructor
        if  pd.notnull(row["Driver"]):  # Driver
            if selected_drivers < max_drivers and cost + row["Cost"] <= budget:
                x.append(1)  # Select this driver
                cost += row["Cost"]
                val += row["Points"]
                selected_drivers += 1
            else:
                x.append(0)  # Don't select this driver
        else:  # Constructor
            if selected_constructors < max_constructors and cost + row["Cost"] <= budget:
                x.append(1)  # Select this constructor
                cost += row["Cost"]
                val += row["Points"]
                selected_constructors += 1
            else:
                x.append(0)  # Don't select this constructor

        # If we hit the budget limit, stop adding further items
        if cost > budget:
            break

    all_items["x"] = x
    return all_items

final_drivers = drivers.copy()
final_constructors = constructors.copy()

print(greedy_f1(final_drivers, final_constructors, budget, max_drivers, max_constructors))

                   Driver  Points  Cost      Constructor  VC_Ratio  x
3                     NaN      61   8.2             Haas  7.439024  1
1                     NaN     136  23.3         Mercedes  5.836910  1
0                     NaN     172  30.6          McLaren  5.620915  0
5                     NaN      32   8.0     Racing Bulls  4.000000  0
6            Esteban Ocon      32   8.1              NaN  3.950617  1
5            Lance Stroll      33   9.3              NaN  3.548387  1
2                     NaN      86  25.4  Red Bull Racing  3.385827  0
0            Lando Norris     100  29.6              NaN  3.378378  1
9          Oliver Bearman      22   6.7              NaN  3.283582  1
1   Andrea Kimi Antonelli      61  19.3              NaN  3.160622  0
8         Nico Hulkenberg      22   7.6              NaN  2.894737  1
2          George Russell      60  21.4              NaN  2.803738  0
4                     NaN      36  13.5         Williams  2.666667  0
6                   